In [18]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from urllib.parse import urlparse


In [19]:
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(options=options)
driver.maximize_window()

urls = []
base_url = "https://www.olx.com.br"

# Define o número de páginas a serem percorridas
num_paginas = 100

for i in range(1, num_paginas + 1):
    url = f"{base_url}/imoveis/venda/estado-sp?lis=home_body_search_bar_1001&o={i}"
    driver.get(url)

    # Aguarda a página carregar completamente
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a[data-ds-component='DS-NewAdCard-Link']"))
        )
    except Exception as e:
        print(f"Erro ao carregar a página {i}: {e}")
        continue

    # Faz scroll para carregar mais anúncios, se necessário
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Coletar os links dos anúncios
    soup = BeautifulSoup(driver.page_source, "html.parser")
    anuncios = soup.find_all("a", {"data-ds-component": "DS-NewAdCard-Link"})

    for anuncio in anuncios:
        link = anuncio["href"]
        if link.startswith("/"):
            link = base_url + link
        urls.append(link)

# Fechar o navegador Selenium
driver.quit()

# Exibir os links coletados
for link in urls:
    print(link)

print(f"Total de links coletados: {len(urls)}")

https://sp.olx.com.br/regiao-de-ribeirao-preto/imoveis/apartamento-02-quartos-com-suite-quintal-privativo-vaga-coberta-1355385578
https://sp.olx.com.br/regiao-de-ribeirao-preto/imoveis/apartamento-02-quartos-com-suite-quintal-privativo-vaga-coberta-1355385578
https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/oportunidade-unica-apartamento-de-alto-padrao-com-preco-imperdivel-1383631684
https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/oportunidade-unica-apartamento-de-alto-padrao-com-preco-imperdivel-1383631684
https://sp.olx.com.br/regiao-de-sorocaba/imoveis/apartamento-edificio-giuseppe-tatui-1346957238
https://sp.olx.com.br/regiao-de-sorocaba/imoveis/apartamento-edificio-giuseppe-tatui-1346957238
https://sp.olx.com.br/regiao-de-sorocaba/imoveis/apartamento-2-quartos-2-banheiros-1-suite-1373656578
https://sp.olx.com.br/regiao-de-sorocaba/imoveis/apartamento-2-quartos-2-banheiros-1-suite-1373656578
https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/imovel-para-venda-possui-60-metros-quadra

In [20]:
urls = list(set(urls))
print(len(urls))

5334


In [ ]:
df = pd.DataFrame()

def deve_processar_url(url):
    """Verifica se a URL deve ser processada (não começa com www)"""
    parsed = urlparse(url)
    return not parsed.netloc.startswith('www.')

# Funções auxiliares
def extrair_acabamentos(texto):
    padrao = r"porcelanato|ceramica|cerâmica|cerâmico|cerâmicos|gesso|antiderrapante|planejado|tábua"
    if isinstance(texto, str):
        encontrados = re.findall(padrao, texto.lower())
        return ", ".join(set(encontrados)) if encontrados else None
    return None

def extrair_tipo_imovel(texto):
    padrao = r"casa|apartamento|sobrado|fazenda"
    if isinstance(texto, str):
        encontrados = re.findall(padrao, texto.lower())
        return ", ".join(set(encontrados)) if encontrados else None
    return None

scraper = cloudscraper.create_scraper()
urls = list(set(urls))  # Remove duplicatas

for url in urls:
    #Pula as urls que iniciam com www
    time.sleep(1)  # Atraso para evitar bloqueio
    print(f"Processando URL: {url}")
    if not deve_processar_url(url):
        print(f"Pulando URL não permitida: {url}")
        continue
    
    response = scraper.get(url, headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"})

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        dados_imovel = {"Link": url}

        # Busca Localização
        location = soup.find("div", {"id": "location"})
        if location:
            endereco_texto = location.get_text(" ", strip=True)
            cep = re.findall(r"\d{5}-?\d{3}", endereco_texto)
            cep = cep[-1] if cep else None
            endereco_limpo = endereco_texto.replace("Localização", "")\
                                        .replace("Exibir no mapa", "")\
                                        .replace(cep, "")\
                                        .strip()
            endereco_limpo = re.sub(r'[,\s-]+$', '', endereco_limpo).strip()
            dados_imovel.update({
                "Endereço": endereco_limpo,
                "CEP": cep
            })

        # Busca Valor
        div_valor_mae = soup.find("div", {"id": "price-box-container"}) 
        if div_valor_mae:
            div_valor = div_valor_mae.find("div", {"class": "olx-d-flex olx-fd-column"}) if div_valor_mae else None
            if div_valor:
                valor_texto = div_valor.get_text(strip=True)
                valor_limpo = re.sub(r'[^\d,]', '', valor_texto).replace(',', '.').strip()
                dados_imovel["Valor Imóvel"] = valor_limpo if valor_limpo else None


        # Busca características do imóvel
        div_principal = soup.find("div", {"class": "ad__sc-wuor06-0 hfcCRQ"})
        if div_principal:
            divs_caracteristicas = div_principal.find_all("div", recursive=False)
            
            for div in divs_caracteristicas:
                texto = div.get_text(strip=True)
                if "Área construída" in texto:
                    dados_imovel["Área construída"] = re.search(r"(\d+)", texto).group(1) if re.search(r"(\d+)", texto) else None
                elif "Quartos" in texto:
                    dados_imovel["Quartos"] = texto.replace("Quartos", "").strip()
                elif "Banheiros" in texto:
                    dados_imovel["Banheiros"] = texto.replace("Banheiros", "").strip()
                elif "Vagas na garagem" in texto:
                    dados_imovel["Vagas na garagem"] = texto.replace("Vagas na garagem", "").strip()

        # Descrição do imóvel
        descricao = soup.find("div", {"id": "description-title"})
        descricao_texto = descricao.get_text(" ", strip=True) if descricao else None
        dados_imovel["Descrição"] = descricao_texto


        # Adiciona dados extras
        if descricao:
            desc_texto = descricao.get_text(" ", strip=True)
            dados_imovel["Acabamentos"] = extrair_acabamentos(desc_texto)
            dados_imovel["Tipo Imóvel"] = extrair_tipo_imovel(desc_texto)

        df = pd.concat([df, pd.DataFrame([dados_imovel])], ignore_index=True)
        print(dados_imovel)


Processando URL: https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/apartamento-duplex-cobertura-em-sao-paulo-1391381414
{'Link': 'https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/apartamento-duplex-cobertura-em-sao-paulo-1391381414', 'Endereço': 'Rua Carlos Tiago Pereira Bosque da Saúde, São Paulo, SP', 'CEP': '04150080', 'Valor Imóvel': '2700000', 'Quartos': '4', 'Banheiros': '5 Ou Mais', 'Vagas na garagem': '5 Ou Mais', 'Descrição': 'Apartamento Duplex/Cobertura em São Paulo Código do anúncio: 3807\n\nCobertura residencial para Venda\nBosque da Saúde, São Paulo\nCom 340M², 4  suítes sendo 1 suite máster,  sala para 3 ambientes, lareira, terraço com ótima, cozinha com planejados, piso em madeira, área de serviço com banheiro e dormitórios para funcionários + 5 vagas de garagem. Próximo a Linha de Metro, Padaria, Shopping, Colégio, Clube, Parque, Academia, Supermercados e Hortifrúti.', 'Acabamentos': 'planejado', 'Tipo Imóvel': 'apartamento'}
Processando URL: https://sp.olx.com.br/sao-p

In [23]:
df.to_excel("imoveis.xlsx", index=False)